In [1]:
import enum
import typing
from dataclasses import dataclass,field
import numpy as np

import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import cmasher as cmr

import networkx as nx
from collections import Counter

In [137]:
exstr = """#.#####################
#.......#########...###
#######.#########.#.###
###.....#.>.>.###.#.###
###v#####.#v#.###.#.###
###.>...#.#.#.....#...#
###v###.#.#.#########.#
###...#.#.#.......#...#
#####.#.#.#######.#.###
#.....#.#.#.......#...#
#.#####.#.#.#########v#
#.#...#...#...###...>.#
#.#.#v#######v###.###v#
#...#.>.#...>.>.#.###.#
#####v#.#.###v#.#.###.#
#.....#...#...#.#.#...#
#.#########.###.#.#.###
#...###...#...#...#.###
###.###.#.###v#####v###
#...#...#.#.>.>.#.>.###
#.###.###.#.###.#.#v###
#.....###...###...#...#
#####################.#
"""

exmap = [line for line in exstr.splitlines()]
exstart = (0,1)
exend = (len(exmap)-1,len(exmap[0])-2)

exendpoints = {exstart,exend}

In [138]:
with open("input23.txt") as f:
    labymap = [line for line in f.read().splitlines()]
N = len(labymap)
M = len(labymap[0])

In [139]:
start = (0,1)
end = (N-1,M-2)

endpoints = {start,end}

# Part 1

In [172]:
directions = {(1,0):"v",(-1,0):"^",(0,1):">",(0,-1):"<"}

def notwallneighbours(y,x,labyrinth):
    paths = set()
    for (dy,dx),slope in directions.items():
        try:
            if labyrinth[y+dy][x+dx]!= "#":
                paths.add((y+dy,x+dx))
        except IndexError: pass
    return paths

def directneighbours(y,x,labyrinth):
    paths = set()
    for (dy,dx),slope in directions.items():
        try:
            if labyrinth[y+dy][x+dx] in (".",slope):
                paths.add((y+dy,x+dx))
        except IndexError: pass
    return paths

def searchneighbours(y,x,labyrinth,start,end,slippery=True):
    longdistanceneighbours = {}
    used = set()
    if slippery:
        trailheads = directneighbours
    else:
        trailheads = notwallneighbours
    for dn in trailheads(y,x,labyrinth):
        if dn not in used:
            path = {(y,x),dn}
            avantgarde = dn
            while nextnodes:=notwallneighbours(*avantgarde,labyrinth)-(path|used):
                # print(avantgarde,nextnodes)
                if (start in nextnodes) or (end in nextnodes):
                    avantgarde = nextnodes.pop()
                    longdistanceneighbours[avantgarde]= {"weight":len(path)}
                elif len(nextnodes)>1: #new node
                    longdistanceneighbours[avantgarde]= {"weight":len(path)-1}
                    break
                else:
                    avantgarde = nextnodes.pop()
                    path.add(avantgarde)
            used.add(dn)
    return longdistanceneighbours

def buildgraph(labyrinth,start,end,slippery=True):
    N = len(labyrinth)
    M = len(labyrinth[0])
    graph = {}
    graph[start] = searchneighbours(*start,labyrinth,start,end,slippery)
    graph[end] = searchneighbours(*end,labyrinth,start,end,slippery)
    for y in range(1,N-1):
        for x in range(1,M-1):
            if len(notwallneighbours(y,x,labyrinth))>2:
                graph[(y,x)] = searchneighbours(y,x,labyrinth,start,end,slippery)
    return nx.DiGraph(graph)


In [173]:
exgraph = buildgraph(exmap,exstart,exend,slippery=True)

In [174]:
sorted([nx.path_weight(exgraph,p,"weight") for p in nx.all_simple_paths(exgraph,exstart,exend)])

[74, 82, 82, 86, 90, 94]

In [175]:
labygraph = buildgraph(labymap,start,end,slippery=True)

In [176]:
max([nx.path_weight(labygraph,p,"weight") for p in nx.all_simple_paths(labygraph,start,end)])

2174

# Part 2

In [178]:
labygraph2 = buildgraph(labymap,start,end,slippery=False)
exgraph2 = buildgraph(exmap,exstart,exend,slippery=False)

In [179]:
sorted([nx.path_weight(exgraph2,p,"weight") for p in nx.all_simple_paths(exgraph2,exstart,exend)])

[74, 82, 82, 86, 90, 94, 110, 118, 118, 126, 150, 154]

In [180]:
max([nx.path_weight(labygraph2,p,"weight") for p in nx.all_simple_paths(labygraph2,start,end)])

6506